
# Monitoria — Interpolação e Polinômios

Este notebook implementa funções de **Cálculo Numérico**:

- Interpolação linear
- Interpolação quadrática
- Polinômios de Lagrange
- Polinômios de Newton
- Polinômios de Gregory–Newton


In [ ]:

import math
from typing import List, Sequence, Tuple, Union

Numero = Union[int, float]


## Funções auxiliares

In [ ]:

def para_lista_float(valores: Sequence[Numero]) -> List[float]:
    return [float(v) for v in valores]

def validar_tamanho_igual(x: Sequence[Numero], y: Sequence[Numero]):
    if len(x) != len(y):
        raise ValueError("As listas x e y devem ter o mesmo tamanho.")

def ordenar_por_x(x: Sequence[Numero]) -> List[int]:
    return sorted(range(len(x)), key=lambda i: x[i])


## Interpolação Linear

In [ ]:

def interpolacao_linear_pontos(x0: float, y0: float, x1: float, y1: float, x: float) -> float:
    if x0 == x1:
        raise ValueError("x0 e x1 não podem ser iguais.")
    return y0 + (y1 - y0) * ((x - x0) / (x1 - x0))

def interpolacao_linear(x_dados: Sequence[Numero], y_dados: Sequence[Numero], x: Union[Numero, Sequence[Numero]]) -> Union[float, List[float]]:
    x_dados = para_lista_float(x_dados)
    y_dados = para_lista_float(y_dados)
    validar_tamanho_igual(x_dados, y_dados)
    ordem = ordenar_por_x(x_dados)
    x_ordenado = [x_dados[i] for i in ordem]
    y_ordenado = [y_dados[i] for i in ordem]

    def interp(xq: float) -> float:
        for i in range(len(x_ordenado) - 1):
            if x_ordenado[i] <= xq <= x_ordenado[i + 1]:
                return interpolacao_linear_pontos(x_ordenado[i], y_ordenado[i], x_ordenado[i + 1], y_ordenado[i + 1], xq)
        raise ValueError("x fora do intervalo de interpolação.")

    if isinstance(x, (int, float)):
        return interp(float(x))
    else:
        return [interp(float(xi)) for xi in x]


## Interpolação Quadrática

In [ ]:

def interpolacao_quadratica(x_dados: Sequence[Numero], y_dados: Sequence[Numero], x: float) -> float:
    x_dados = para_lista_float(x_dados)
    y_dados = para_lista_float(y_dados)
    validar_tamanho_igual(x_dados, y_dados)
    if len(x_dados) != 3:
        raise ValueError("São necessários exatamente 3 pontos para interpolação quadrática.")

    x0, x1, x2 = x_dados
    y0, y1, y2 = y_dados

    a = y0 / ((x0 - x1) * (x0 - x2)) + y1 / ((x1 - x0) * (x1 - x2)) + y2 / ((x2 - x0) * (x2 - x1))
    b = -y0 * (x1 + x2) / ((x0 - x1) * (x0 - x2)) - y1 * (x0 + x2) / ((x1 - x0) * (x1 - x2)) - y2 * (x0 + x1) / ((x2 - x0) * (x2 - x1))
    c = y0 * x1 * x2 / ((x0 - x1) * (x0 - x2)) + y1 * x0 * x2 / ((x1 - x0) * (x1 - x2)) + y2 * x0 * x1 / ((x2 - x0) * (x2 - x1))

    return a * x ** 2 + b * x + c


## Polinômio de Lagrange

In [ ]:

def lagrange(x_dados: Sequence[Numero], y_dados: Sequence[Numero], x: float) -> float:
    x_dados = para_lista_float(x_dados)
    y_dados = para_lista_float(y_dados)
    validar_tamanho_igual(x_dados, y_dados)

    n = len(x_dados)
    resultado = 0.0

    for i in range(n):
        termo = y_dados[i]
        for j in range(n):
            if i != j:
                termo *= (x - x_dados[j]) / (x_dados[i] - x_dados[j])
        resultado += termo

    return resultado


## Polinômio de Newton (Diferenças Divididas)

In [ ]:

def tabela_diferencas_divididas(x_dados: Sequence[Numero], y_dados: Sequence[Numero]) -> List[List[float]]:
    n = len(x_dados)
    tabela = [[0.0 for _ in range(n)] for _ in range(n)]
    for i in range(n):
        tabela[i][0] = y_dados[i]
    for j in range(1, n):
        for i in range(n - j):
            tabela[i][j] = (tabela[i + 1][j - 1] - tabela[i][j - 1]) / (x_dados[i + j] - x_dados[i])
    return tabela

def newton(x_dados: Sequence[Numero], y_dados: Sequence[Numero], x: float) -> float:
    tabela = tabela_diferencas_divididas(x_dados, y_dados)
    n = len(x_dados)
    resultado = tabela[0][0]
    produto = 1.0
    for i in range(1, n):
        produto *= (x - x_dados[i - 1])
        resultado += tabela[0][i] * produto
    return resultado


## Polinômio de Gregory–Newton

In [ ]:

def tabela_diferencas_finitas(y: Sequence[Numero]) -> List[List[float]]:
    n = len(y)
    tabela = [list(y)]
    for i in range(1, n):
        anterior = tabela[-1]
        nova = [anterior[j + 1] - anterior[j] for j in range(len(anterior) - 1)]
        tabela.append(nova)
    return tabela

def gregory_newton(x_dados: Sequence[Numero], y_dados: Sequence[Numero], x: float) -> float:
    h = x_dados[1] - x_dados[0]
    tabela = tabela_diferencas_finitas(y_dados)
    n = len(x_dados)
    s = (x - x_dados[0]) / h
    resultado = tabela[0][0]
    termo = 1.0
    for i in range(1, n):
        termo *= (s - i + 1) / i
        resultado += termo * tabela[i][0]
    return resultado


## Teste das funções

In [ ]:

import math

x_pontos = [0, 1, 2]
y_pontos = [1, 3, 2]

print("Linear:", interpolacao_linear(x_pontos, y_pontos, 1.5))
print("Quadrática:", interpolacao_quadratica(x_pontos, y_pontos, 1.5))
print("Lagrange:", lagrange(x_pontos, y_pontos, 1.5))
print("Newton:", newton(x_pontos, y_pontos, 1.5))
print("Gregory–Newton:", gregory_newton(x_pontos, y_pontos, 1.5))
